In [1]:
import pandas as pd
import seaborn as sns
import urllib.request
import gzip
import qgrid

import os.path as osp

/usr/local/lib/python3.4/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
# download data here if it doesn't exist already
net_file_path = './dm3.droVir3.net.gz'
net_file_url = 'http://hgdownload.soe.ucsc.edu/goldenPath/dm3/vsDroVir3/dm3.droVir3.net.gz'

if not osp.exists(net_file_path):
    print('downloading synteny file')
    urllib.request.urlretrieve(net_file_url, net_file_path)

In [3]:
# Net File Format Reference: https://genome.ucsc.edu/goldenPath/help/net.html

data = []

def StringIsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False


with gzip.open(net_file_path, 'r') as netfile:
    for i, line in enumerate(netfile):
        fields = line.decode('utf-8').strip().split()
        
        if fields[0] == 'net':
            current_chromosome = fields[1]
            continue
        
        # lines prefixed with 'gap' are not matching
        lineIsNotSynteny = fields[0] != 'fill'
        
        if lineIsNotSynteny:
            continue
    
        # These are attributes that are mandatory and fixed    
        _, tstart, tsize, qname, plusminus, qstart, qsize = fields[:7]
        
        # There are extra attributes that are not always specified/are optional
        optional_attributes = {}


        for optional_attribute_name in fields[7::2]:
            index_of_attribute = fields.index(optional_attribute_name)
            attribute_value = fields[index_of_attribute + 1]
            optional_attributes[optional_attribute_name] = int(attribute_value) \
                                                            if StringIsInt(attribute_value)\
                                                            else attribute_value
        
        attributes_table  = {
                 'melano_chromosome': current_chromosome, 
                 'melano_start': int(tstart),
                 'melano_size': int(tsize),
                 'virilis_chromosome': qname,
                 'relative_orientation': plusminus,
                 'virilis_start': int(qstart),
                 'virilis_size': int(qsize)}
        
        
        attributes_table.update(optional_attributes)
        
        data.append(attributes_table)

        
my_data_frame = pd.DataFrame(data)

In [4]:
qgrid.show_grid(my_data_frame)

In [5]:
syntenic_query = 'melano_size > 10000 and virilis_size > 10000 and type == "syn" '\
                 'and melano_chromosome == "chr2L"'

big_syntenic_blocks = my_data_frame.query(syntenic_query)\
    .sort_values('score', ascending=False)

In [15]:
if not osp.exists('syntenic_blocks.csv'):
    print('saving syntenic blocks to csv')
    big_syntanic_blocks.to_csv('syntenic_blocks.csv')
    

In [7]:
qgrid.show_grid(big_syntenic_blocks)

In [14]:
# long range contact stuff
long_range_contact_url = 'http://www.cell.com/cms/attachment/2007964571/2030739776/mmc3.xls'
long_range_contact_path = './long_range_contacts.xls'

if not osp.exists(long_range_contact_path):
    print('downloading long range contacts')
    urllib.request.urlretrieve(long_range_contact_url, long_range_contact_path)

In [9]:
long_range_contacts = pd.read_excel(long_range_contact_path, header=3)
qgrid.show_grid(long_range_contacts)

In [10]:
# on 2l, the contact points
chromosome_2L_points = long_range_contacts.query('Chr == "2L"')

In [11]:
# looped areas (should count from outside of sticking area)
start_stop_points = ['Xstart', 'Xend', 'Ystart', 'Yend']
forward_loops = chromosome_2L_points['Yend'] - chromosome_2L_points['Xstart']
backward_loops = chromosome_2L_points['Xend'] - chromosome_2L_points['Ystart']

is_forward_loop = forward_loops > 0
is_backward_loop = ~is_forward_loop

loop_lengths = forward_loops * is_forward_loop + backward_loops * is_backward_loop
chromosome_2L_points['loop_length'] = loop_lengths
qgrid.show_grid(chromosome_2L_points)

/usr/local/lib/python3.4/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
filtered_points = chromosome_2L_points.query('loop_length < 3e6') 
qgrid.show_grid(filtered_points)

In [13]:
loops = pd.DataFrame()

loops['loop_start'] = filtered_points[start_stop_points].min(axis=1)
loops['loop_stop'] = filtered_points[start_stop_points].max(axis=1)

# def overlap(start1, end1, start2, end2):
#     """Does the range (start1, end1) overlap with (start2, end2)?"""
#     return end1 >= start2 and end2 >= start1

def overlap(a,b,c,d):
    r = 0 if a==c and b==d else min(b,d)-max(a,c)
    if r >= 0: 
        return r
    else:
        return 0

def overlapper(m_start, m_end):
    def overlap_inner(row):
        return overlap(row['loop_start'], row['loop_stop'], m_start, m_end)
    
    return overlap_inner
    
for row in big_syntenic_blocks.itertuples():
    # calculate coverage with loops
    index = row[0]
    melano_size = row[4]
    melano_start = row[5]
    melano_end = melano_start + melano_size
    
    overlap_function = overlapper(melano_start, melano_end)
    
    coverage = loops.apply(overlap_function, axis=1) / melano_size
    
    covering_loop = coverage.argmax()
    
    print('block {} coverage {} covering loop {}'.format(index, coverage.max(), covering_loop))
    

block 607 coverage 1.0 covering loop 17
block 1985 coverage 0.0 covering loop 12
block 1541 coverage 1.0 covering loop 159
block 2384 coverage 1.0 covering loop 120
block 321 coverage 1.0 covering loop 12
block 1215 coverage 1.0 covering loop 87
block 1239 coverage 1.0 covering loop 87
block 1414 coverage 1.0 covering loop 87
block 1745 coverage 0.0 covering loop 12
block 907 coverage 0.0 covering loop 12
block 2824 coverage 0.022754525622318215 covering loop 26
block 1054 coverage 1.0 covering loop 87
block 253 coverage 1.0 covering loop 12
block 271 coverage 1.0 covering loop 12
block 1191 coverage 1.0 covering loop 87
block 1029 coverage 1.0 covering loop 87
block 1148 coverage 1.0 covering loop 87
block 1079 coverage 1.0 covering loop 87
block 1799 coverage 0.0 covering loop 12
block 3155 coverage 0.0 covering loop 12
block 1980 coverage 0.0 covering loop 12
block 433 coverage 1.0 covering loop 17
block 1965 coverage 0.0 covering loop 12
block 1971 coverage 0.0 covering loop 12
blo